# 第4回 その2: 重回帰分析
その1:単回帰分析では気温データのみを用いてアイスの売上を予測しようとしていました。  
今回は，気温に加えて降水量と湿度のデータも用いてアイスの売り上げを予測してみます。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/04"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/04/
# 現在のフォルダの中身を表示
%ls

`icecream2.csv`というデータが表示されていることを確認してください。

## ステップ1: 重回帰分析
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`icecream2.csv` を読み込みます。  
このデータは，`icecream.csv`に記録されていた東京の平均気温，アイス支出額に加えて，降水量と平均湿度も加わったデータです。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('icecream2.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

month = csv_data.loc[:, '年月']
# numpy用データ(ndarray型) として抽出する。
# 説明変数：平均気温，降水量，湿度を得る
X = csv_data.loc[:, '平均気温(℃)':'平均湿度(％)'].to_numpy()
# 目的変数：アイス支出額を得る
y = csv_data.loc[:, 'アイス支出額(円)'].to_numpy()

# 説明変数のサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

説明変数は「平均気温」，「降水量の合計」，「平均湿度」の3つなので，  
説明変数`X` は 3次元ベクトルが 36個(12か月x3年) 入ったデータとなります。

説明変数の各次元と，目的変数をプロットしてみます。

In [ ]:
ylabel_set = ['Temperature [deg]', 'Rainfall [mm]', 'Humidity [%]']
plt.figure(figsize=(15,20))
x = np.arange(np.size(month))

# 説明変数のプロット
for n in range(3):
  plt.subplot(4,1,n+1)
  plt.plot(x, X[:,n], color='b', label=ylabel_set[n])
  # 補助目盛線の描画
  plt.grid(axis='x')
  # x軸の目盛りラベルの設定
  plt.xticks(x, month, rotation=90)
  plt.ylabel(ylabel_set[n])
  plt.legend()

# 目的変数のプロット
plt.subplot(4,1,4)
plt.plot(x, y, color='b', label='Expending for icecream [yen]')
plt.grid(axis='x')
plt.xticks(x, month, rotation=90)
plt.ylabel('Expending for icecream [yen]')
plt.legend()
plt.show()

重回帰分析を行う関数を以下に定義します。  
以下の関数では，単回帰分析にも対応するような実装をしています。  

In [ ]:
# 重回帰分析を行う
def mlr(x, y):
  '''
      x: 説明変数 [サンプル数, 次元数]の行列
      y: 目的変数 [サンプル数] のベクトル
  '''
  # 単回帰分析にも対応するための処理
  if x.ndim == 1:
    X = np.reshape(x, (-1,1))
  else:
    X = np.array(x)

  # 入力された説明変数のサンプル数と次元数を得る
  (num_samples, num_dimensions) = np.shape(X)
  
  # 切片（バイアス）に相当する項を説明変数に追加
  # x が
  # x11 x12
  # x21 x22
  # という行列の場合，X は
  # x11 x12 1
  # x21 x22 1
  # という行列になる
  X = np.concatenate((X, np.ones((num_samples, 1))), axis=1)

  # 重回帰分析の式
  XX = np.dot(X.T, X)
  XXX = np.dot(np.linalg.inv(XX), X.T)
  a = np.dot(XXX, y)
  
  # a の最後の要素は切片に相当
  b = a[-1]
  # a の最後を除く要素は各説明変数の係数に相当
  a = a[:-1]
  
  return (a, b)

関数 `mlr` を用いて重回帰分析を実行します。

In [ ]:
(a, b) = mlr(X, y)

print('Result of multiple linear regression')
print('y = (%f * temperature) + (%f * rainfall) + (%f * humidity) + %f' % (a[0], a[1], a[2], b))

predicted_mlr = np.dot(X, a) + b

重回帰分析の結果によると，気温に対する回帰係数は正の値，降水量に対する回帰係数は0に近い負の値，湿度に対する回帰係数は負の値になりました。    
この回帰式においては，気温が高い程アイスはよく売れ，一方湿度が高いとあまり売れなくなります。また降水量はアイス売上とはほとんど関係しないということになります。

比較のため，気温だけを説明変数に用いた場合の単回帰分析も実行します。  
用いる関数は 04_01_single_linear_regression.ipynb の `slr` と異なりますが，結果は04_01_single_linear_regression.ipynbと同じになります。  

In [ ]:
# X の0次元目(=気温)のみを抽出して mlr に入力する。
(a, b) = mlr(X[:, 0], y)

print('Result of sinlge linear regression')
print('y = %f * temperature + %f' % (a, b))

predicted_slr = a * X[:,0] + b

単回帰分析（気温のみを使用）と重回帰分析（気温，降水量，湿度を使用）のアイス売上推定結果をプロットして比べてみます。

In [ ]:
x = np.arange(np.size(month))
plt.figure(figsize=(13,5))
plt.plot(x, y, label='True value', color='b')
plt.plot(x, predicted_slr, label='Single linear regression', color='r')
plt.plot(x, predicted_mlr, label='Multiple linear regression', color='g')
plt.grid(axis='x')
plt.xticks(x, month, rotation=90)
plt.ylabel('Expending for icecream [yen]')
plt.legend()
plt.show()

推定誤差と相関係数を比較してみます。

In [ ]:
# 相関係数を求める関数
def calc_corr_coef(x, y):
  '''
      x, y: 相関を求めたいデータ
  '''
  mu_x = np.mean(x)
  mu_y = np.mean(y)
  std_x = np.std(x)
  std_y = np.std(y)

  corr_coef = np.mean((x - mu_x) * (y - mu_y)) / (std_x * std_y)

  return corr_coef

# 単回帰分析の誤差・相関係数
mse = np.mean((y - predicted_slr)*(y - predicted_slr))
rmse = np.sqrt(mse)
corr_coef = calc_corr_coef(predicted_slr, y)
print('Single linear regression')
print('  MSE = %f' % (mse))
print('  RMSE = %f' % (rmse))
print('  Correlation coefficent = %f' % (corr_coef))
print('----------------------------')
# 重回帰分析の誤差・相関係数
mse = np.mean((y - predicted_mlr)*(y - predicted_mlr))
rmse = np.sqrt(mse)
corr_coef = calc_corr_coef(predicted_mlr, y)
print('Multiple linear regression')
print('  MSE = %f' % (mse))
print('  RMSE = %f' % (rmse))
print('  Correlation coefficent = %f' % (corr_coef))

比較すると，単回帰分析よりも推定誤差が小さくなり，また相関係数も高くなっていることが分かりました。  

とは言え本当にこの重回帰分析結果が正しいかどうかは，今回用いたデータの収集期間（2017～2019年）以外のデータ（テストデータ）に対しても誤差を計算するなどして，きちんと検証する必要がある点には注意してください。